1) Where did environment go?
Into SweepJob, look for EnvironmentDefinition class
2) How do I turn an AmlCompute into a ComputeConfiguration
Create ComputeConfiguration based off of compute. Pass in compute.arm_id as target
3) What args are required for Parameterized command?
Code, Compute, and Environment
3a) Is "base_path" where the script is located?
No, base_path is internal.
4) Deploy???
No, job outputs yet (incl in Sept)

Thing I can do:
Create SweepJob yaml and azure.ml.job.load(<yaml>)

In [1]:
# Make workspace
%env AZURE_EXTENSION_DIR=/home/schrodinger/automl/sdk-cli-v2/src/cli/src
%env AZURE_ML_CLI_PRIVATE_FEATURES_ENABLED=true

env: AZURE_EXTENSION_DIR=/home/schrodinger/automl/sdk-cli-v2/src/cli/src
env: AZURE_ML_CLI_PRIVATE_FEATURES_ENABLED=true


# Prepare workspace, experiment, and compute

In [2]:
import azure.ml
from azure.ml import MLClient

SUBSCRIPTION_ID = '381b38e9-9840-4719-a5a0-61d9585e1e91'
RESOURCE_GROUP_NAME = 'lesam_centraluseuap_rg'
WORKSPACE_NAME = "lesam-master"
experiment_name = "MySdkV2SweepJobExperiment"
CLUSTER_NAME = "cpu-cluster"

client = MLClient(SUBSCRIPTION_ID, RESOURCE_GROUP_NAME, default_workspace_name=WORKSPACE_NAME)

client

In [3]:
# Make Compute
# choose a name for your cluster
from azure.ml.entities import AmlCompute

try:
    compute_target = client.compute.get(CLUSTER_NAME)
    print('Found existing compute target.')
except Exception:
    print('Creating a new compute target...')
    compute = AmlCompute(
        name=CLUSTER_NAME, size="STANDARD_NC6",
        min_instances=0, max_instances=4,
        idle_time_before_scale_down=120
    )

    # create the cluster
    client.compute.create(compute)
    compute_target = client.compute.get(CLUSTER_NAME)

compute_target

Found existing compute target.


AmlCompute({'name': 'cpu-cluster', 'id': '/subscriptions/381b38e9-9840-4719-a5a0-61d9585e1e91/resourceGroups/lesam_centraluseuap_rg/providers/Microsoft.MachineLearningServices/workspaces/lesam-master/computes/cpu-cluster', 'description': None, 'tags': {}, 'properties': {}, 'base_path': './', 'location': 'centraluseuap', 'type': 'amlcompute', 'enable_public_ip': False, 'resource_id': None, 'provisioning_state': 'Succeeded', 'provisioning_errors': None, 'created_on': None, 'size': 'STANDARD_D2_V2', 'min_instances': 0, 'max_instances': 4, 'idle_time_before_scale_down': 120.0, 'identity_type': None, 'user_assigned_identities': None, 'admin_username': 'azureuser', 'admin_password': None, 'ssh_key_value': None, 'vnet_name': None, 'subnet': None, 'priority': 'Dedicated'})

# Prepare Sweep Job

In [4]:
# Prepare Early Termination Policy
from azure.ml._restclient.v2021_03_01_preview.models import MedianStoppingPolicy
early_termination_policy = MedianStoppingPolicy(evaluation_interval=1, delay_evaluation=5)

In [5]:
# Prepare search space
from azure.ml.entities import Choice, Uniform, Randint
search_space = {
    "y1": Choice(["foo", "bar", "baz"]),
    "y2": Uniform(0, 1),
    "y3": Randint(999)
}

In [6]:
# Make SweepJob
from azure.ml.entities import SweepJob, ParameterizedCommand, Code, ComputeConfiguration, Environment
from azure.ml._restclient.v2021_03_01_preview.models import Objective

objective = Objective(goal="Maximize", primary_metric="Accuracy")
code = Code(local_path = ".", name = "foo", version = 3)
compute = ComputeConfiguration(target = compute_target.id)
environment = client.environments.get(name='AzureML-TensorFlow-2.3-GPU', version=17)

In [7]:
trial = ParameterizedCommand(
    command = 'python example_script.py --x1 {search_space.y1} --x2 {search_space.y2} --x3 {search_space.y3}',
    code = code,
    compute = compute,
    environment = environment,
)

In [8]:
sweep_job = SweepJob(
    algorithm = "Random",
    search_space = search_space,
    objective = objective,
    trial = trial,
    early_termination = early_termination_policy,
    max_concurrent_trials = 2,
    max_total_trials = 2,
    timeout_minutes = 60,
)

In [9]:
# Submit
created_job = client.jobs.create_or_update(sweep_job)
created_job

Uploading train-hyperparameter-tune-deploy:   0%|                   | 0.00/129k [00:00<?, ?B/s]


HttpResponseError: Operation returned an invalid status 'Request is invalid and/or missing fields.'